In [14]:
import os
import numpy as np
from keras.preprocessing import image
import cv2
from PIL import Image
from os import path
import shutil, copy
from copy import deepcopy
from image_cropper import *
from collections import defaultdict

In [15]:
def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    # print(str(len(file_list)))
    return file_list

def mk_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path

data_type = '.jpg'
TARGET_SIZE = 384

In [16]:
data_img_path_train = '../../datasets/bcn/train/images/'

data_save_path = '../../datasets/processed/bcn/roi_square_cropped/'
data_save_path_train = mk_dir(data_save_path + 'train/')

In [17]:
bcn_classes = ['actinic keratosis', 'basal cell carcinoma', 'dermatofibroma', 'melanoma', 'nevus', 'seborrheic keratosis', 'solar lentigo', 'squamous cell carcinoma', 'vascular lesion']

HamLabels = defaultdict(lambda: "NONE")
HamLabels['actinic keratosis'] = "AKIEC"
HamLabels['basal cell carcinoma'] = "BCC"
HamLabels['dermatofibroma'] = "DF"
HamLabels['melanoma'] = "MEL"
HamLabels['nevus'] = "NV"
HamLabels['seborrheic keratosis'] = "BKL"
HamLabels['solar lentigo'] = "BKL"
HamLabels['squamous cell carcinoma'] = "NONE"
HamLabels['vascular lesion'] = "VASC"

In [18]:
train_metadata = pd.read_csv("../../datasets/bcn/train_metadata_isic.csv")
target_metadata_train = deepcopy(train_metadata)
target_metadata_train['HamMappedLabel'] = HamLabels["DEFAULT"]

In [19]:
for bcn_label in bcn_classes:
    target_metadata_train.loc[target_metadata_train.diagnosis == bcn_label, 'HamMappedLabel'] = HamLabels[bcn_label]

target_metadata = target_metadata_train[target_metadata_train.HamMappedLabel != "NONE"]
target_metadata.to_csv(f"{data_save_path}target_metadata_train.csv", index=False)

In [20]:
i=0
for image_name in list(target_metadata.isic_id):
    i+=1
    image_name += data_type
    if i % 1000 == 0:
        print('Processing Img {idx}: {image_name}'.format(idx=i, image_name=image_name))
    
    org_img = np.asarray(image.load_img(data_img_path_train + image_name))
    # roi square crop
    gray_image = cv2.cvtColor(org_img, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_image, 60, 255, 0)
    try:
        rect = obtain_bb(thresh)

        x,y,w,h = rect
        int_ratio = intensity_ratio(gray_image, x, y, w, h)

        if float(int_ratio) > 1.1:
            print('Image {} cropped successfully.'.format(image_name))
            org_img = org_img[y:y+h, x:x+w]
    except:
        print('Image {} was not found'.format(image_name))
        continue
   
    # resize
    org_img = cv2.resize(org_img, (TARGET_SIZE, TARGET_SIZE), interpolation = cv2.INTER_AREA)
   
    cropImg = Image.fromarray(org_img)
    cropImg.save(path.join(data_save_path_train, image_name[:-4] + '.jpg'))


Image ISIC_0053470.jpg cropped successfully.
Image ISIC_0053479.jpg cropped successfully.
Image ISIC_0053484.jpg cropped successfully.
Image ISIC_0053506.jpg cropped successfully.
Image ISIC_0053524.jpg cropped successfully.
Image ISIC_0053528.jpg cropped successfully.
Image ISIC_0053529.jpg cropped successfully.
Image ISIC_0053531.jpg cropped successfully.
Image ISIC_0053533.jpg cropped successfully.
Image ISIC_0053541.jpg cropped successfully.
Image ISIC_0053551.jpg cropped successfully.
Image ISIC_0053552.jpg cropped successfully.
Image ISIC_0053555.jpg cropped successfully.
Image ISIC_0053564.jpg cropped successfully.
Image ISIC_0053572.jpg cropped successfully.
Image ISIC_0053574.jpg cropped successfully.
Image ISIC_0053609.jpg cropped successfully.
Image ISIC_0053626.jpg cropped successfully.
Image ISIC_0053643.jpg cropped successfully.
Image ISIC_0053644.jpg cropped successfully.
Image ISIC_0053648.jpg cropped successfully.
Image ISIC_0053659.jpg cropped successfully.
Image ISIC